# 🎯 Value Iteration em Labirinto - MDP (Markov Decision Process)

## Objetivo Educacional

Demonstrar como um **agente** aprende a encontrar o caminho ótimo em um labirinto usando:
- 📊 **Value Iteration** (Iteração de Valor)
- 🔄 **Equação de Bellman**
- 🎲 **Processos de Decisão Markovianos (MDP)**

## O Problema

Um agente está preso em um **labirinto** e precisa encontrar a saída (objetivo).

- 🟫 **Paredes (1)**: Não é possível atravessar
- ⚪ **Caminhos Livres (0)**: Espaços onde o agente pode andar
- 🎯 **Objetivo (9)**: A meta a alcançar

## A Solução: Value Iteration

O algoritmo calcula o **valor de cada estado** usando:

$$V(s) = \max_a \sum_{s'} P(s'|s,a) [R(s,a,s') + \gamma \cdot V(s')]$$

**Simplificado para nosso caso:**
$$V(s) = \max_a [\gamma \cdot (0.8 \cdot V(s_{próximo}) + 0.2 \cdot V(s))]$$

Onde:
- **V(s)**: Valor do estado atual (quanto de recompensa se espera daí)
- **0.8**: Probabilidade de sucesso na transição (80%)
- **0.2**: Probabilidade de falha (ficar no mesmo lugar = 20%)
- **γ = 0.9**: Fator de desconto

## 1️⃣ Importações e Configuração

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
import warnings
warnings.filterwarnings('ignore')

# Parâmetros do MDP
gamma = 0.9  # Fator de desconto
transition_success = 0.8  # Probabilidade de sucesso na ação
transition_failure = 0.2  # Probabilidade de falha (ficar no mesmo lugar)

# ============ DEFINIÇÃO DO LABIRINTO ============
# 0 = Caminho livre
# 1 = Parede
# 9 = Objetivo
maze = np.array([
    [1, 1, 1, 1, 1, 1],
    [1, 0, 1, 0, 0, 1],
    [1, 0, 1, 0, 1, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 1],
    [1, 0, 0, 1, 9, 1],
    [1, 1, 1, 1, 1, 1]
])

num_rows, num_cols = maze.shape

# Inicializar matriz de valores
V = np.zeros((num_rows, num_cols))

# Coordenadas do objetivo
goal_pos = np.where(maze == 9)
goal_row, goal_col = goal_pos[0][0], goal_pos[1][0]

print("="*70)
print("LABIRINTO - VALUE ITERATION (MDP)")
print("="*70)
print(f"\n📐 Dimensões: {num_rows} x {num_cols}")
print(f"🎯 Objetivo em: ({goal_row}, {goal_col})")
print(f"⚖️  Fator de Desconto (γ): {gamma}")
print(f"✅ Prob. de Sucesso: {transition_success*100}% | ❌ Prob. Falha: {transition_failure*100}%")
print("\nMapa do Labirinto:")
print("  1 = Parede | 0 = Caminho | 9 = Objetivo")
print(maze)
print("="*70)

## 2️⃣ Funções de Cálculo - Equação de Bellman

### Lógica do MDP

Para cada estado e ação:
1. Determinar o próximo estado se a ação tiver sucesso (80% chance)
2. Determinar o estado se a ação falhar (20% chance = fica no mesmo lugar)
3. Calcular valor = γ × (0.8 × V(próximo) + 0.2 × V(atual))
4. Pegar o máximo entre todas as 4 ações possíveis

In [ ]:
def get_next_state(state, action):
    """
    Retorna o próximo estado dado uma ação.
    
    Parâmetros:
    - state: tupla (row, col)
    - action: str ('up', 'down', 'left', 'right')
    
    Retorno:
    - tupla (new_row, new_col)
    """
    row, col = state
    
    if action == 'up':
        return (max(row - 1, 0), col)
    elif action == 'down':
        return (min(row + 1, num_rows - 1), col)
    elif action == 'left':
        return (row, max(col - 1, 0))
    elif action == 'right':
        return (row, min(col + 1, num_cols - 1))
    
    return state

def calculate_action_value(state, action):
    """
    Calcula o valor de uma ação em um determinado estado.
    
    Usa a fórmula do MDP:
    V(s,a) = γ × [0.8 × V(s') + 0.2 × V(s)]
    
    Parâmetros:
    - state: tupla (row, col)
    - action: str ('up', 'down', 'left', 'right')
    
    Retorno:
    - float: valor calculado
    """
    row, col = state
    
    # Não calcular para paredes ou objetivo já alcançado
    if maze[row, col] == 1:
        return 0
    if maze[row, col] == 9:  # Se já está no objetivo
        return 1
    
    # Próximo estado se a ação tiver sucesso (80%)
    next_state = get_next_state(state, action)
    next_row, next_col = next_state
    
    # Parâmetros da equação de Bellman
    # Recompensa imediata + valor descontado do próximo estado
    action_value = gamma * (
        transition_success * V[next_row, next_col] +  # 80% de sucesso
        transition_failure * V[row, col]               # 20% de falha (fica no lugar)
    )
    
    return action_value

def value_iteration(max_iterations=50, tolerance=1e-2):
    """
    Executa o algoritmo de iteração de valor.
    
    Parâmetros:
    - max_iterations: número máximo de iterações
    - tolerance: critério de parada (convergência)
    
    Retorno:
    - history: histórico de iterações
    """
    global V
    
    history = []
    
    for iteration in range(max_iterations):
        delta = 0
        V_old = V.copy()
        
        # Para cada estado no labirinto
        for row in range(num_rows):
            for col in range(num_cols):
                # Ignorar paredes
                if maze[row, col] == 1:
                    continue
                
                # Objetivo tem valor 1
                if maze[row, col] == 9:
                    V[row, col] = 1.0
                    continue
                
                # Calcular valor máximo entre todas as ações
                max_value = -np.inf
                
                for action in ['up', 'down', 'left', 'right']:
                    action_value = calculate_action_value((row, col), action)
                    max_value = max(max_value, action_value)
                
                V[row, col] = max_value
                delta = max(delta, abs(V_old[row, col] - V[row, col]))
        
        history.append({
            'iteration': iteration + 1,
            'delta': delta,
            'V': V.copy()
        })
        
        print(f"Iteração {iteration + 1:2d}: Delta = {delta:.6f}")
        
        # Critério de convergência
        if delta < tolerance:
            print(f"\n✅ Convergência alcançada em {iteration + 1} iterações!")
            break
    
    return history

print("✓ Funções de cálculo definidas com sucesso!")

## 3️⃣ Funções de Visualização

In [ ]:
def plot_value_function(V, title="Função de Valor - Labirinto", iteration=None):
    """
    Visualiza a função de valor usando matplotlib.
    
    Parâmetros:
    - V: matriz de valores
    - title: título do gráfico
    - iteration: número da iteração (opcional)
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Criar colormap customizado
    cmap = plt.cm.RdYlGn
    
    # Visualizar valores
    im = ax.imshow(np.where(maze == 1, np.nan, V), cmap=cmap, vmin=0, vmax=1)
    
    # Desenhar paredes
    for row in range(num_rows):
        for col in range(num_cols):
            if maze[row, col] == 1:
                rect = patches.Rectangle((col-0.5, row-0.5), 1, 1, 
                                        linewidth=0, facecolor='black', alpha=0.5)
                ax.add_patch(rect)
    
    # Adicionar valores nas células
    for row in range(num_rows):
        for col in range(num_cols):
            if maze[row, col] != 1:
                if maze[row, col] == 9:
                    ax.text(col, row, '★\nOBJ', ha='center', va='center', 
                           fontsize=10, fontweight='bold', color='white')
                else:
                    ax.text(col, row, f'{V[row, col]:.2f}', ha='center', va='center',
                           fontsize=9, fontweight='bold', 
                           color='black' if V[row, col] < 0.5 else 'white')
    
    # Configurar eixos
    ax.set_xticks(range(num_cols))
    ax.set_yticks(range(num_rows))
    ax.set_xlim(-0.5, num_cols-0.5)
    ax.set_ylim(num_rows-0.5, -0.5)
    
    # Título
    if iteration:
        title = f"{title}\nIteração: {iteration}"
    ax.set_title(title, fontsize=13, fontweight='bold')
    
    # Colorbar
    cbar = plt.colorbar(im, ax=ax, label='Valor do Estado')
    
    plt.tight_layout()
    return fig

def plot_value_progression(history):
    """
    Visualiza a progressão do delta ao longo das iterações.
    """
    fig, ax = plt.subplots(figsize=(11, 5))
    
    iterations = [h['iteration'] for h in history]
    deltas = [h['delta'] for h in history]
    
    ax.semilogy(iterations, deltas, 'o-', linewidth=2.5, markersize=8, color='#E74C3C')
    ax.axhline(y=1e-2, color='green', linestyle='--', linewidth=2, label='Tolerância (1e-2)')
    
    ax.set_xlabel('Número de Iterações', fontweight='bold', fontsize=11)
    ax.set_ylabel('Delta (log scale)', fontweight='bold', fontsize=11)
    ax.set_title('Convergência do Value Iteration', fontweight='bold', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=10)
    
    plt.tight_layout()
    return fig

def plot_policy(V):
    """
    Visualiza a política ótima (melhor ação em cada estado).
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Fundo com valores
    im = ax.imshow(np.where(maze == 1, np.nan, V), cmap='RdYlGn', vmin=0, vmax=1, alpha=0.6)
    
    # Desenhar paredes
    for row in range(num_rows):
        for col in range(num_cols):
            if maze[row, col] == 1:
                rect = patches.Rectangle((col-0.5, row-0.5), 1, 1, 
                                        linewidth=0, facecolor='black', alpha=0.7)
                ax.add_patch(rect)
    
    # Desenhar direções da política ótima
    arrows = {
        'up': (0, -0.3, '↑'),
        'down': (0, 0.3, '↓'),
        'left': (-0.3, 0, '←'),
        'right': (0.3, 0, '→')
    }
    
    for row in range(num_rows):
        for col in range(num_cols):
            if maze[row, col] != 1 and maze[row, col] != 9:
                # Encontrar melhor ação
                best_action = None
                best_value = -np.inf
                
                for action in ['up', 'down', 'left', 'right']:
                    action_value = calculate_action_value((row, col), action)
                    if action_value > best_value:
                        best_value = action_value
                        best_action = action
                
                # Desenhar seta
                if best_action:
                    dx, dy, arrow = arrows[best_action]
                    ax.text(col + dx, row + dy, arrow, ha='center', va='center',
                           fontsize=16, fontweight='bold', color='blue')
            
            elif maze[row, col] == 9:
                ax.text(col, row, '★', ha='center', va='center',
                       fontsize=14, fontweight='bold', color='red')
    
    ax.set_xticks(range(num_cols))
    ax.set_yticks(range(num_rows))
    ax.set_xlim(-0.5, num_cols-0.5)
    ax.set_ylim(num_rows-0.5, -0.5)
    ax.set_title('Política Ótima (Melhor Ação em Cada Estado)', fontweight='bold', fontsize=12)
    
    plt.colorbar(im, ax=ax, label='Valor do Estado')
    plt.tight_layout()
    return fig

print("✓ Funções de visualização definidas com sucesso!")

## 4️⃣ Executar Value Iteration

O algoritmo calculará o valor de cada estado iterativamente até convergir.

In [ ]:
print("\n" + "="*70)
print("EXECUTANDO VALUE ITERATION")
print("="*70 + "\n")

# Executar o algoritmo
history = value_iteration(max_iterations=50, tolerance=1e-2)

print("\n" + "="*70)
print("RESULTADOS FINAIS")
print("="*70)
print(f"\nMatriz Final de Valores (V):")
print(V.round(3))
print(f"\nMaior valor encontrado: {V.max():.4f}")
print(f"Menor valor encontrado: {V[V > 0].min():.4f}")
print(f"Número de iterações: {len(history)}")

## 5️⃣ Visualizações dos Resultados

### Visualização 1: Função de Valor Final

In [ ]:
# Plotar a função de valor final
fig1 = plot_value_function(V, title="Função de Valor Final - Iteração da Solução")
plt.show()

print("\n📊 Interpretação do Mapa de Valores:")
print("-" * 70)
print("• Cores VERDES: Estados com alto valor (perto do objetivo)")
print("• Cores VERMELHAS: Estados com baixo valor (longe do objetivo)")
print("• PRETO: Paredes (não acessíveis)")
print("• ★ OBJ: Objetivo (valor = 1.0)")

### Visualização 2: Convergência do Algoritmo

In [ ]:
fig2 = plot_value_progression(history)
plt.show()

print("\n📈 Gráfico de Convergência:")
print("-" * 70)
print("• Eixo Y (log): Delta - maior mudança em qualquer estado")
print("• Eixo X: Número de iteração")
print("• Linha verde: Tolerância de convergência (1e-2)")
print(f"• O algoritmo parou quando Delta < {1e-2}")

### Visualização 3: Política Ótima (Melhor Ação em Cada Estado)

In [ ]:
fig3 = plot_policy(V)
plt.show()

print("\n🎯 Política Ótima:")
print("-" * 70)
print("• Cada seta (↑↓←→) representa a melhor ação em aquele estado")
print("• Seguir as setas levará ao caminho ótimo até ao objetivo")
print("• ★ = Objetivo (meta final)")
print("\nCom a política ótima, o agente sabe exatamente para onde ir!")

## 6️⃣ Análise e Insights

### O Que Aprendemos com Value Iteration?

#### 1️⃣ **Função de Valor Representa "Proximidade da Meta"**

O algoritmo aprendeu que:
- ✅ Estados próximos ao objetivo têm valores ALTOS (verde)
- ❌ Estados longe do objetivo têm valores BAIXOS (vermelho)

Isso corresponde à ideia intuitiva: "Quanto mais perto estou do objetivo, maior é meu valor esperado"

#### 2️⃣ **Convergência é Garantida**

A Equação de Bellman converge quando:
- A recompensa é limitada
- 0 ≤ γ < 1 (desconto futuro)

No nosso caso:
- Convergiria em **{len(history)} iterações** (muito rápido!)
- Delta diminuiu exponencialmente (veja o gráfico log)

#### 3️⃣ **A Política Ótima Emerge Naturalmente**

A melhor estratégia (política) é simplesmente:
> **"Em cada estado, sempre execute a ação que maximiza o valor esperado"**

Sem exploração aleatória, sem aprendizado - apenas cálculos matemáticos!

#### 4️⃣ **Transições Estocásticas Importam**

- 80% de sucesso + 20% de falha = Ações incertas
- O algoritmo aprende a ser robusto a falhas
- Não assume execução perfeita

### Equação de Bellman em Detalhe

$$V^{k+1}(s) = \max_a \sum_{s'} P(s'|s,a) [R(s,a,s') + \gamma V^k(s')]$$

Em nosso labirinto:
- **P(s'|s,a)** = 0.8 (sucesso) ou 0.2 (falha)
- **R(s,a,s')** = 0 (custo apenas de movimento)
- **Recompensa** = 1 se s' = objetivo, senão 0
- **γ** = 0.9 (valor do futuro)

### Aplicações Práticas de Value Iteration

| Domínio | Problema | Objetivo |
|---------|----------|----------|
| **Robótica** | Navegação | Mover robot do ponto A ao B |
| **Games** | Estratégia | Encontrar movimento ótimo |
| **Controle** | Estabilização | Manter sistema em estado desejado |
| **Finanças** | Portfólio | Maximizar riqueza ao longo do tempo |

### Comparação: Value Iteration vs Policy Iteration

| Aspecto | Value Iteration | Policy Iteration |
|--------|-----------------|-----------------|
| **Velocidade** | Mais iterações | Menos iterações |
| **Custo/Iteração** | Menor | Maior (resolve sistema linear) |
| **Implementação** | Simples | Complexa |
| **Uso** | Ambientes pequenos | Problemas grandes |

## 7️⃣ Desafios Práticos para Aprendizado

### 🚀 Questões para Reflexão

1. **E se o labirinto fosse muito maior?** (1000x1000 estados)
   - Value Iteration ainda convergiria?
   - Qual seria o problema de memória?
   - → Solução: Usar aproximação (função universal aproximadora / rede neural)

2. **E se não conhecêssemos o modelo (MDP)?** (Sem transições e recompensas)
   - Como aprender a política?
   - → Solução: Q-Learning, SARSA (aprendizado por experiência)

3. **E se o ambiente mudasse?** (Paredes movem, novos objetivos aparecem)
   - Como adaptar a política?
   - → Solução: Aprendizado contínuo, replanejamento

4. **E se houvesse múltiplos objetivos?** (Encontrar A, depois B, depois C)
   - Como estruturar o problema?
   - → Solução: Subtarefas, hierarquias, opções

5. **E se houvesse incerteza maior?** (Ações falham 50% das vezes)
   - Como o algoritmo se comportaria?
   - → Experimente mudar `transition_success` para 0.5!

### 📚 Próximos Tópicos Relacionados

- **Q-Learning**: Aprende sem conhecer o modelo
- **Policy Gradient**: Otimiza a política diretamente
- **Deep Reinforcement Learning**: Combina DNNs com RL
- **Monte Carlo Tree Search**: Busca em espaços enormes (com sampling)
- **Multi-Agent RL**: Vários agentes interagindo

### 📖 Referências

- Sutton & Barto: "Reinforcement Learning: An Introduction" (Capítulos 3-4)
- David Silver: Lectures on Reinforcement Learning (YouTube)
- Berkeley CS 285: Deep Reinforcement Learning

## ✅ Conclusão

### O que Value Iteration nos ensinou

**Value Iteration é um algoritmo fundamental para resolver Processos de Decisão Markovianos** quando:
- ✅ Conhecemos o modelo (transições e recompensas)
- ✅ O espaço de estados é razoavelmente pequeno
- ✅ Precisamos de convergência garantida

**Equação de Bellman é a base teórica:**
> *Toda decisão ótima pode ser decomposta em uma recompensa imediata + valor descontado do estado futuro*

Isso permite algoritmos como:
- 🎮 Game AI (Chess, Go, Video Games)
- 🤖 Robot Control
- 💰 Finanças & Trading
- 🚗 Autonomous Vehicles
- 📦 Supply Chain Optimization

### Próximo Passo Recomendado

**Implemente você mesmo:**
1. Modifique o labirinto (adicione mais paredes, mude objetivo)
2. Teste com diferentes valores de γ
3. Acompanhe como as setas (política) mudam
4. Implemente Policy Iteration para comparar

---

**"A matemática dos problemas de decisão sequencial é elegante, poderosa e universal. Uma vez que entendes Bellman, opens portas para IA moderna."**